In [1]:
# Include project path to available custom class at jupyter
import os
import sys
sys.path.insert(0, os.path.abspath('/home/vanessa/PycharmProjects/RecurrentNetworks/'))

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load libs standard python and custom
import numpy as np
import pandas as pd
import datetime

from network_model.custom_ensemble import CustomEnsemble
from network_model.model_class import ModelClass
from utils.experiment_processes import ExperimentProcesses
from utils.log import Log
import utils.definition_network as dn

import datetime

# LAYERS
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


Using TensorFlow backend.


<h2> Running the main LSTM topologies for multilabel dataset </h2>
<p>
    <ul> When reviewing the experiments, there were some gaps that motivated the tests below:
    <li> <b> 1.0 Train, validate and test experiment lstm_exp14_L3_N64_B20_E96 </b>: experiment with the best result multilabel dataset.
        <ol><li> <b> Test lstm_exp14_L3_N64_B20_E96 + SMHD _ * _ 2640 </b> </li>
            <li> <b> Test lstm_exp14_L3_N64_B20_E96 + SMHD _ * _ 1760 </b> </li>
        </ol>
    </li>
    <li> <b> 2.0 Train, validate and test experiment lstm_exp9_var_L3_N16_B40_E32_D0.2 </b>: it has already been tested with multi-label dataset for the best configuration using Glove6B static embeddings. However, this same model and dataset and personalized word embeddings were tested, generating the set of tests below:
        <ol><li> <b> Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD all users (SMHD _ * _ 1760) </b>, generated with algorithms (skipgram, cbow, glove) </li>
            <li> <b> Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD all users (SMHD _ * _ 2640) </b>, generated with algorithms (skipgram, cbow, glove) </li>
            <li> <b> Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD 3 pathologies (best result dataset between tests 2.1 and 2.2) </b>, generated with algorithms (skipgram, cbow, glove) </li>
            <li> <b> Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + Glove6B, glorot x lecun kernel initializer </b> </li>
        </ol>
    </li>
    </ul>
</p>

In [3]:
### Checks composition of the used dataset train
file_path = "/home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/"
dataset_train = pd.read_pickle(file_path + str("SMHD_train_2640.df"))
dataset_train.groupby('label').size()

label
anxiety               440
anxiety,depression    440
control               880
depression            440
dtype: int64

In [4]:
### Checks composition of the used dataset valid
dataset_valid = pd.read_pickle(file_path + str("SMHD_validation_2640.df"))
dataset_valid.groupby('label').size()

label
anxiety               440
anxiety,depression    440
control               880
depression            440
dtype: int64

In [5]:
### Checks composition of the used dataset test
dataset_test = pd.read_pickle(file_path + str("SMHD_test_2640.df"))
dataset_test.groupby('label').size()

label
anxiety               440
anxiety,depression    440
control               880
depression            440
dtype: int64

In [6]:
dataset = pd.concat([dataset_train, dataset_valid, dataset_test])
dataset.describe()

,label,texts
count,6600,6600
unique,4,6600
top,control,"['We get new celebrities. ', 'Wolf Children. V..."
freq,2640,1


### 1.0 Train, validate and test experiment lstm_exp14_L3_N64_B20_E96

In [7]:
def generate_lstm_exp14(exp, name_model):
    exp.pp_data.vocabulary_size = 5000
    exp.pp_data.embedding_size = 300
    exp.pp_data.max_posts = 1750
    exp.pp_data.max_terms_by_post = 300
    exp.pp_data.binary_classifier = True
    exp.pp_data.format_input_data = dn.InputData.POSTS_LIST
    exp.pp_data.remove_stopwords = False
    exp.pp_data.delete_low_tfid = False
    exp.pp_data.min_df = 0
    exp.pp_data.min_tf = 0
    exp.pp_data.random_posts = False
    exp.pp_data.random_users = False
    exp.pp_data.tokenizing_type = 'WE'
    exp.pp_data.word_embedding_custom_file = ''    
    exp.pp_data.use_embedding = dn.UseEmbedding.NONE
    exp.pp_data.embedding_type = dn.EmbeddingType.NONE
    exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL
    exp.pp_data.type_prediction_label= dn.TypePredictionLabel.MULTI_LABEL_CATEGORICAL

    exp.use_custom_metrics = False
    exp.use_valid_set_for_train = True
    exp.valid_split_from_train_set = 0.0
    exp.imbalanced_classes = False

    we_file_name = 'ET_' + str(exp.pp_data.embedding_type.value) + '_UE_' + str(exp.pp_data.use_embedding.value) +\
                                 '_EF_' + 'glove6B300d_glorot'
    
    lstm = ModelClass(1)
    lstm.loss_function = 'binary_crossentropy'
    lstm.optmizer_function = 'adam'
    lstm.use_embedding_pre_train = exp.pp_data.use_embedding
    lstm.embed_trainable = False

    # Train
    neuronios_by_layer = [64]
    epochs = [96]
    batch_sizes = [20]
    dropouts = [0.2]

    np.random.seed(dn.SEED)
    time_ini_rep = datetime.datetime.now()

    x_train, y_train, x_valid, y_valid, num_words, embedding_matrix = exp.pp_data.load_data()
    exp.set_period_time_end(time_ini_rep, 'Load data')

    for neuronios in neuronios_by_layer:
        for batch_size in batch_sizes:
            for epoch in epochs:
                for dropout in dropouts:
                    lstm.epochs = epoch
                    lstm.batch_size = batch_size
                    lstm.patience_train = epoch/2
                    data_dim = exp.pp_data.max_terms_by_post
                    timesteps = exp.pp_data.max_posts

                    exp.experiment_name = name_model + 'lstm_exp14_L3' + '_N' + str(neuronios) + '_B' +\
                                          str(batch_size) + '_E' + str(epoch) + '_D' + str(dropout) + '_' +\
                                          we_file_name

                    lstm.model = Sequential()
                    lstm.model.add(
                            LSTM(neuronios, activation='tanh', dropout=dropout, recurrent_dropout=dropout, 
                                 return_sequences=True, stateful=True, 
                                 batch_input_shape=(batch_size, timesteps, data_dim),
                                 name='dense_1_' + name_model))
                    lstm.model.add(
                            LSTM(neuronios, activation='tanh', dropout=dropout, recurrent_dropout=dropout,
                                 return_sequences=True, stateful=True,
                                 name='dense_2_' + name_model))
                    lstm.model.add(LSTM(neuronios, activation='tanh', dropout=dropout, recurrent_dropout=dropout,
                                        stateful=True,
                                        name='dense_3_' + name_model))
                    lstm.model.add(Dense(3, activation='sigmoid',
                                         name='dense_4_' + name_model))

                    time_ini_exp = datetime.datetime.now()
                    exp.generate_model_hypeparams(lstm, x_train, y_train, x_valid, y_valid, embedding_matrix)
                    exp.set_period_time_end(time_ini_exp, 'Total experiment')

    del x_train, y_train, x_valid, y_valid, num_words, embedding_matrix

    # Test
    exp.pp_data.load_dataset_type = dn.LoadDataset.TEST_DATA_MODEL
    np.random.seed(dn.SEED)
    time_ini_rep = datetime.datetime.now()
    x_test, y_test = exp.pp_data.load_data()
    exp.set_period_time_end(time_ini_rep, 'Load data')

    for neuronios in neuronios_by_layer:
        for batch_size in batch_sizes:
            for epoch in epochs:
                for dropout in dropouts:
                    exp.experiment_name = name_model + 'lstm_exp14_L3' + '_N' + str(neuronios) + '_B' +\
                                          str(batch_size) + '_E' + str(epoch) + '_D' + str(dropout) + '_' +\
                                          we_file_name
                    lstm.epochs = epoch
                    lstm.batch_size = batch_size
                    lstm.patience_train = epoch/2

                    lstm.model = exp.load_model(dn.PATH_PROJECT + exp.experiment_name + '.h5')
                    exp.save_geral_configs()
                    exp.save_summary_model(lstm.model)
                    exp.predict_samples(lstm, x_test, y_test)
    
    del x_test, y_test, lstm, exp

#### 1.1 Test lstm_exp14_L3_N64_B20_E96 + SMHD_*_2640

In [8]:
print('Initializer experiment 1 (model SMHD_ml_gl_2640)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label')

exp = ExperimentProcesses('lstm_exp14_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=2640, subdirectory="anx_dep_multilabel")

generate_lstm_exp14(exp, 'exp14_ml_gl_2640')

Initializer experiment 1 (model SMHD_ml_gl_2640)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_2_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_train_2640.df
Load data - Ini: 2020-03-08 13:49:08	End: 2020-03-08 13:50:19	Total: 0:01:11.455724




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Training using single GPU or CPU..

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2200 samples, validate on 2200 samples
Epoch 1/96
2200/2200 [==============================] - 266s 121ms/step - loss: 0.6764 - acc: 0.5995 - val_loss: 0.6736 - val_acc: 0.6000
Epoch 2/96
2200/2200 [==============================] - 258s 117ms/step - loss: 0.6741 - acc

Epoch 41/96
2200/2200 [==============================] - 262s 119ms/step - loss: 0.6733 - acc: 0.6000 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 42/96
2200/2200 [==============================] - 262s 119ms/step - loss: 0.6732 - acc: 0.6000 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 43/96
2200/2200 [==============================] - 263s 120ms/step - loss: 0.6734 - acc: 0.6000 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 44/96
2200/2200 [==============================] - 263s 120ms/step - loss: 0.6734 - acc: 0.6000 - val_loss: 0.6730 - val_acc: 0.6000
Epoch 45/96
2200/2200 [==============================] - 262s 119ms/step - loss: 0.6733 - acc: 0.6000 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 46/96
2200/2200 [==============================] - 261s 119ms/step - loss: 0.6733 - acc: 0.6000 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 47/96
2200/2200 [==============================] - 263s 119ms/step - loss: 0.6732 - acc: 0.6000 - val_loss: 0.6730 - val_acc: 0.6000
Epoch 48/96
2200/2200 [====

Load data - Ini: 2020-03-08 20:49:20	End: 2020-03-08 20:49:50	Total: 0:00:29.492355
2200/2200 [==============================] - 55s 25ms/step


#### 1.2 Test lstm_exp14_L3_N64_B20_E96 + SMHD_*_1760

In [9]:
print('Initializer experiment 1.1 (model SMHD_ml_gl_1760)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_1760 multi-label')

exp = ExperimentProcesses('lstm_exp14_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=1760, subdirectory="anx_dep_multilabel")

generate_lstm_exp14(exp, 'exp14_ml_gl_1760')

Initializer experiment 1.1 (model SMHD_ml_gl_1760)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_1760 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_1760_VS_5000_TF_0_DF_0_RSW_F_IT_2_RP_F
Preprocess data...
Generate tokenizing
Load data - Ini: 2020-03-08 20:51:02	End: 2020-03-08 20:53:31	Total: 0:02:29.144501
Training using single GPU or CPU..
Train on 1760 samples, validate on 1760 samples
Epoch 1/96
1760/1760 [==============================] - 209s 119ms/step - loss: 0.6560 - acc: 0.5816 - val_loss: 0.6517 - val_acc: 0.5833
Epoch 2/96
1760/1760 [==============================] - 207s 117ms/step - loss: 0.6520 - acc: 0.5841 - val_loss: 0.6506 - val_acc: 0.5833
Epoch 3/96
1760/1760 [==============================] - 209s 119ms/step - loss: 0.6527 - acc: 0.5824 - val_loss: 0.6502 - val_acc: 0.5833
Epoch 4/96
1760/1760 [==============================] - 206s 117ms/step - loss: 0.6525 - acc: 0.5712 - 

### 2.0 Train, validate and test experiment lstm_exp9_var_L3_N16_B40_E32_D0.2

In [8]:
def generate_model(exp, name_model, we_file_name, kernel_function='glorot_uniform'):
    exp.pp_data.vocabulary_size = 5000

    exp.pp_data.embedding_size = 300
    exp.pp_data.max_posts = 1750
    exp.pp_data.max_terms_by_post = 300
    exp.pp_data.binary_classifier = True
    exp.pp_data.format_input_data = dn.InputData.POSTS_ONLY_TEXT
    exp.pp_data.remove_stopwords = False
    exp.pp_data.delete_low_tfid = False
    exp.pp_data.min_df = 0
    exp.pp_data.min_tf = 0
    exp.pp_data.random_posts = False
    exp.pp_data.random_users = False
    exp.pp_data.tokenizing_type = 'WE'
    exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL
    exp.pp_data.type_prediction_label= dn.TypePredictionLabel.MULTI_LABEL_CATEGORICAL

    exp.use_custom_metrics = False
    exp.use_valid_set_for_train = True
    exp.valid_split_from_train_set = 0.0
    exp.imbalanced_classes = False

    lstm = ModelClass(1)
    lstm.loss_function = 'binary_crossentropy'
    lstm.optmizer_function = 'adam'
    lstm.epochs = 15
    lstm.batch_size = 32
    lstm.patience_train = 10
    lstm.use_embedding_pre_train = exp.pp_data.use_embedding
    lstm.embed_trainable = (lstm.use_embedding_pre_train == (dn.UseEmbedding.RAND or dn.UseEmbedding.NON_STATIC))

    neuronios_by_layer = [16]
    epochs = [32]
    batch_sizes = [40]
    dropouts = [0.2]

    np.random.seed(dn.SEED)

    time_ini_rep = datetime.datetime.now()
    x_train, y_train, x_valid, y_valid, num_words, embedding_matrix = exp.pp_data.load_data()
    exp.set_period_time_end(time_ini_rep, 'Load data')

    for neuronios in neuronios_by_layer:
        for batch_size in batch_sizes:
            for epoch in epochs:
                for dropout in dropouts:
                    lstm.epochs = epoch
                    lstm.batch_size = batch_size
                    lstm.patience_train = epoch / 2
                    exp.experiment_name = name_model + '_lstm_exp9_var_L3' + '_N' + str(neuronios) + '_B' +\
                                          str(batch_size) + '_E' + str(epoch) + '_D' + str(dropout) + '_' +\
                                          we_file_name

                    lstm.model = Sequential()
                    lstm.model.add(Embedding(exp.pp_data.vocabulary_size, exp.pp_data.embedding_size,
                                             trainable=lstm.embed_trainable, name='emb_' + name_model))
                    lstm.model.add(LSTM(neuronios, kernel_initializer=kernel_function,
                                        activation='tanh', dropout=dropout, recurrent_dropout=dropout,
                                        return_sequences=True, name='dense_1_' + name_model))
                    lstm.model.add(LSTM(neuronios, kernel_initializer=kernel_function,
                                        activation='tanh', dropout=dropout, recurrent_dropout=dropout,
                                        return_sequences=True, name='dense_2_' + name_model))
                    lstm.model.add(LSTM(neuronios, kernel_initializer=kernel_function,
                                        activation='tanh', dropout=dropout, recurrent_dropout=dropout,
                                        name='dense_3_' + name_model))
                    lstm.model.add(Dense(3,
                                         activation='sigmoid',
                                         name='dense_4_' + name_model))

                    time_ini_exp = datetime.datetime.now()
                    exp.generate_model_hypeparams(lstm, x_train, y_train, x_valid, y_valid, embedding_matrix)
                    exp.set_period_time_end(time_ini_exp, 'Total experiment')

    del x_train, y_train, x_valid, y_valid, num_words, embedding_matrix

    # Test
    exp.pp_data.load_dataset_type = dn.LoadDataset.TEST_DATA_MODEL
    np.random.seed(dn.SEED)
    time_ini_rep = datetime.datetime.now()
    x_test, y_test = exp.pp_data.load_data()
    exp.set_period_time_end(time_ini_rep, 'Load data')

    for neuronios in neuronios_by_layer:
        for batch_size in batch_sizes:
            for epoch in epochs:
                for dropout in dropouts:
                    lstm.epochs = epoch
                    lstm.batch_size = batch_size
                    lstm.patience_train = epoch / 2
                    exp.experiment_name = name_model + '_lstm_exp9_var_L3' + '_N' + str(neuronios) + '_B' +\ 
                                          str(batch_size) + '_E' + str(epoch) + '_D' + str(dropout) + '_' +\ 
                                          we_file_name

                    lstm.model = exp.load_model(dn.PATH_PROJECT + exp.experiment_name + '.h5')
                    exp.save_geral_configs()
                    exp.save_summary_model(lstm.model)
                    exp.predict_samples(lstm, x_test, y_test)

    del x_test, y_test, lstm

#### 2.1 Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD all users (SMHD_*_1760)

In [13]:
print('Initializer experiment 1.1 (model SMHD_ml_gl_1760)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_1760 multi-label')

exp = ExperimentProcesses('lstm_exp9_var_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=1760, subdirectory="anx_dep_multilabel")

embedding_types = [dn.EmbeddingType.WORD2VEC_CUSTOM, dn.EmbeddingType.GLOVE_CUSTOM]
use_embeddings = [dn.UseEmbedding.STATIC, dn.UseEmbedding.NON_STATIC]

for embedding_type in embedding_types:
    if embedding_type == dn.EmbeddingType.WORD2VEC_CUSTOM:
        word_embedding_custom_files = ['SMHD-Skipgram-AllUsers-300.bin', 'SMHD-CBOW-AllUsers-300.bin']
    else:
        word_embedding_custom_files = ['SMHD-glove-AllUsers-300.pkl']

    for word_embedding_custom_file in word_embedding_custom_files:
        for use_embedding in use_embeddings:
            exp.pp_data.embedding_type = embedding_type
            exp.pp_data.use_embedding = use_embedding
            exp.pp_data.word_embedding_custom_file = word_embedding_custom_file
            exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL

            we_file_name = 'ET_' + str(embedding_type.value) + '_UE_' + str(use_embedding.value) +\
                           '_EF_' + word_embedding_custom_file.split('.')[0] + '_glove6B300d_glorot'

            generate_model(exp, 'exp9_' + we_file_name[0:13] + we_file_name[18:30] + '_1760', we_file_name)


Initializer experiment 1.1 (model SMHD_ml_gl_1760)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_1760 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_1760_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_train_1760.df
Load data - Ini: 2020-03-09 09:09:44	End: 2020-03-09 09:10:01	Total: 0:00:17.053792
Training using single GPU or CPU..
Train on 1760 samples, validate on 1760 samples
Epoch 1/32
1760/1760 [==============================] - 434s 247ms/step - loss: 0.6666 - acc: 0.5771 - val_loss: 0.6493 - val_acc: 0.5958
Epoch 2/32
1760/1760 [==============================] - 429s 244ms/step - loss: 0.6529 - acc: 0.5811 - val_loss: 0.6484 - val_acc: 0.5858
Epoch 3/32
1760/1760 [==============================] - 422s 240ms/step - loss: 0.6484 - acc: 0.5869 - val_loss: 0.6447 - val_acc: 0.6055
Epoch 4/32
1760/1760 [=======

Epoch 17/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.6066 - acc: 0.6542 - val_loss: 0.6533 - val_acc: 0.5977
Epoch 18/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.5987 - acc: 0.6689 - val_loss: 0.6512 - val_acc: 0.6180
Epoch 19/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.6017 - acc: 0.6644 - val_loss: 0.6491 - val_acc: 0.6144
Epoch 20/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.6026 - acc: 0.6621 - val_loss: 0.6519 - val_acc: 0.6093
Epoch 21/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.5974 - acc: 0.6669 - val_loss: 0.6593 - val_acc: 0.5941
Epoch 22/32
1760/1760 [==============================] - 401s 228ms/step - loss: 0.5947 - acc: 0.6714 - val_loss: 0.6513 - val_acc: 0.6106
Epoch 23/32
1760/1760 [==============================] - 399s 227ms/step - loss: 0.5964 - acc: 0.6680 - val_loss: 0.6480 - val_acc: 0.6186
Epoch 24/32
1760/1760 [====

Epoch 2/32
1760/1760 [==============================] - 424s 241ms/step - loss: 0.6526 - acc: 0.5811 - val_loss: 0.6486 - val_acc: 0.5975
Epoch 3/32
1760/1760 [==============================] - 408s 232ms/step - loss: 0.6488 - acc: 0.5818 - val_loss: 0.6481 - val_acc: 0.5807
Epoch 4/32
1760/1760 [==============================] - 405s 230ms/step - loss: 0.6469 - acc: 0.5949 - val_loss: 0.6463 - val_acc: 0.6129
Epoch 5/32
1760/1760 [==============================] - 399s 227ms/step - loss: 0.6408 - acc: 0.6034 - val_loss: 0.6448 - val_acc: 0.6117
Epoch 6/32
1760/1760 [==============================] - 402s 229ms/step - loss: 0.6335 - acc: 0.6284 - val_loss: 0.6522 - val_acc: 0.5938
Epoch 7/32
1760/1760 [==============================] - 402s 228ms/step - loss: 0.6307 - acc: 0.6263 - val_loss: 0.6433 - val_acc: 0.6150
Epoch 8/32
1760/1760 [==============================] - 395s 225ms/step - loss: 0.6198 - acc: 0.6384 - val_loss: 0.6419 - val_acc: 0.6172
Epoch 9/32
1760/1760 [============

#### 2.2 Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD all users (SMHD_*_2640)

In [9]:
print('Initializer experiment 2.2 (model SMHD_ml_gl_2640)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label')

exp = ExperimentProcesses('lstm_exp9_var_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=2640, subdirectory="anx_dep_multilabel")

embedding_types = [dn.EmbeddingType.WORD2VEC_CUSTOM, dn.EmbeddingType.GLOVE_CUSTOM]
use_embeddings = [dn.UseEmbedding.STATIC, dn.UseEmbedding.NON_STATIC]

for embedding_type in embedding_types:
    if embedding_type == dn.EmbeddingType.WORD2VEC_CUSTOM:
        word_embedding_custom_files = ['SMHD-Skipgram-AllUsers-300.bin', 'SMHD-CBOW-AllUsers-300.bin']
    else:
        word_embedding_custom_files = ['SMHD-glove-AllUsers-300.pkl']

    for word_embedding_custom_file in word_embedding_custom_files:
        for use_embedding in use_embeddings:
            exp.pp_data.embedding_type = embedding_type
            exp.pp_data.use_embedding = use_embedding
            exp.pp_data.word_embedding_custom_file = word_embedding_custom_file
            exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL

            we_file_name = 'ET_' + str(embedding_type.value) + '_UE_' + str(use_embedding.value) +\
                           '_EF_' + word_embedding_custom_file.split('.')[0] + '_glove6B300d_glorot'

            generate_model(exp, 'exp9_' + we_file_name[0:13] + we_file_name[18:30] + '_2640', we_file_name)

Initializer experiment 2.2 (model SMHD_ml_gl_2640)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_train_2640.df
Load Pre-train embeddings
Load data - Ini: 2020-03-10 14:48:34	End: 2020-03-10 14:49:01	Total: 0:00:26.907298




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Training using single GPU or CPU..

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2200 samples, validate on 2200 samples
Epoch 1/32
2200/2200 [==============================] - 537s 244ms/step - loss: 0.6754 - acc: 0.5942 - val_loss: 0.6733 - val_acc: 0.6000
Epoch 2/32
2200/2200 [==============================] - 542s 246

2200/2200 [==============================] - 503s 229ms/step - loss: 0.6227 - acc: 0.6486 - val_loss: 0.6516 - val_acc: 0.6148
Epoch 30/32
2200/2200 [==============================] - 503s 229ms/step - loss: 0.6219 - acc: 0.6479 - val_loss: 0.6570 - val_acc: 0.6091
Epoch 31/32
2200/2200 [==============================] - 502s 228ms/step - loss: 0.6239 - acc: 0.6442 - val_loss: 0.6514 - val_acc: 0.6185
Epoch 32/32
2200/2200 [==============================] - 502s 228ms/step - loss: 0.6237 - acc: 0.6515 - val_loss: 0.6523 - val_acc: 0.6118
Generate Model - Ini: 2020-03-10 22:42:19	End: 2020-03-11 03:12:06	Total: 4:29:47.202715
Total experiment - Ini: 2020-03-10 22:42:17	End: 2020-03-11 03:12:06	Total: 4:29:49.589882
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_test_2640.df
Load data - Ini: 2020-0

Epoch 00028: early stopping
Generate Model - Ini: 2020-03-11 12:29:12	End: 2020-03-11 16:36:10	Total: 4:06:58.517568
Total experiment - Ini: 2020-03-11 12:29:12	End: 2020-03-11 16:36:10	Total: 4:06:58.518239
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_test_2640.df
Load data - Ini: 2020-03-11 16:36:10	End: 2020-03-11 16:36:22	Total: 0:00:11.545395
2200/2200 [==============================] - 72s 33ms/step


#### 2.3 Test lstm_exp9_var_L3_N16_B40_E32_D0.2 + word embeddings SMHD 3 patologias (SMHD_*_2640)

In [10]:
print('Initializer experiment 2.3 (model SMHD_ml_gl_2640)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label')

exp = ExperimentProcesses('lstm_exp9_var_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=2640, subdirectory="anx_dep_multilabel")

embedding_types = [dn.EmbeddingType.WORD2VEC_CUSTOM, dn.EmbeddingType.GLOVE_CUSTOM]
use_embeddings = [dn.UseEmbedding.STATIC, dn.UseEmbedding.NON_STATIC]

for embedding_type in embedding_types:
    if embedding_type == dn.EmbeddingType.WORD2VEC_CUSTOM:
        word_embedding_custom_files = ['SMHD-Skipgram-A-D-ADUsers-300.bin', 'SMHD-CBOW-A-D-ADUsers-300.bin']
    else:
        word_embedding_custom_files = ['SMHD-glove-A-D-ADUsers-300.pkl']

    for word_embedding_custom_file in word_embedding_custom_files:
        for use_embedding in use_embeddings:
            exp.pp_data.embedding_type = embedding_type
            exp.pp_data.use_embedding = use_embedding
            exp.pp_data.word_embedding_custom_file = word_embedding_custom_file
            exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL

            we_file_name = 'ET_' + str(embedding_type.value) + '_UE_' + str(use_embedding.value) +\
                           '_EF_' + word_embedding_custom_file.split('.')[0] + '_glove6B300d_glorot'

            generate_model(exp, 'exp9_' + we_file_name[0:13] + we_file_name[18:30] + '_2640', we_file_name)

Initializer experiment 2.3 (model SMHD_ml_gl_2640)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_train_2640.df
Load Pre-train embeddings
Load data - Ini: 2020-03-11 16:37:52	End: 2020-03-11 16:38:16	Total: 0:00:24.774836
Training using single GPU or CPU..
Train on 2200 samples, validate on 2200 samples
Epoch 1/32
2200/2200 [==============================] - 567s 258ms/step - loss: 0.6773 - acc: 0.5908 - val_loss: 0.6719 - val_acc: 0.6000
Epoch 2/32
2200/2200 [==============================] - 536s 244ms/step - loss: 0.6701 - acc: 0.6003 - val_loss: 0.6612 - val_acc: 0.6029
Epoch 3/32
2200/2200 [==============================] - 531s 241ms/step - loss: 0.6590 - acc: 0.6032 - val_loss: 0.6446 - val_acc: 0.6289
Epo

Epoch 17/32
2200/2200 [==============================] - 529s 240ms/step - loss: 0.6272 - acc: 0.6453 - val_loss: 0.6499 - val_acc: 0.6156
Epoch 18/32
2200/2200 [==============================] - 519s 236ms/step - loss: 0.6240 - acc: 0.6456 - val_loss: 0.6518 - val_acc: 0.6191
Epoch 19/32
2200/2200 [==============================] - 525s 239ms/step - loss: 0.6237 - acc: 0.6491 - val_loss: 0.6562 - val_acc: 0.6124
Epoch 20/32
2200/2200 [==============================] - 524s 238ms/step - loss: 0.6262 - acc: 0.6338 - val_loss: 0.6568 - val_acc: 0.6153
Epoch 21/32
2200/2200 [==============================] - 521s 237ms/step - loss: 0.6261 - acc: 0.6432 - val_loss: 0.6504 - val_acc: 0.6233
Epoch 22/32
2200/2200 [==============================] - 521s 237ms/step - loss: 0.6234 - acc: 0.6514 - val_loss: 0.6456 - val_acc: 0.6232
Epoch 23/32
2200/2200 [==============================] - 518s 235ms/step - loss: 0.6297 - acc: 0.6444 - val_loss: 0.6461 - val_acc: 0.6208
Epoch 24/32
2200/2200 [====

2200/2200 [==============================] - 497s 226ms/step - loss: 0.6631 - acc: 0.6027 - val_loss: 0.6685 - val_acc: 0.6008
Epoch 24/32
2200/2200 [==============================] - 498s 226ms/step - loss: 0.6636 - acc: 0.6011 - val_loss: 0.6691 - val_acc: 0.6033
Epoch 25/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6619 - acc: 0.6073 - val_loss: 0.6574 - val_acc: 0.6111
Epoch 26/32
2200/2200 [==============================] - 496s 225ms/step - loss: 0.6565 - acc: 0.6083 - val_loss: 0.6422 - val_acc: 0.6289
Epoch 27/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6575 - acc: 0.6068 - val_loss: 0.6380 - val_acc: 0.6344
Epoch 28/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6539 - acc: 0.6114 - val_loss: 0.6533 - val_acc: 0.6155
Epoch 29/32
2200/2200 [==============================] - 498s 226ms/step - loss: 0.6541 - acc: 0.6092 - val_loss: 0.6624 - val_acc: 0.6098
Epoch 30/32
2200/2200 [================

#### 2.4 Test lstm_exp9_var_L3_N16_B40_E32_D0.2 +  Glove6B, glorot x lecun kernel initializer

In [10]:
print('Initializer experiment 2.3 (model SMHD_ml_gl_2640)\n'+\
      'Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label')

exp = ExperimentProcesses('lstm_exp9_var_L3')
exp.pp_data.set_dataset_source(dataset_name='SMHD', label_set=['control', 'anxiety', 'depression'],
                               total_registers=2640, subdirectory="anx_dep_multilabel")

use_embeddings = [dn.UseEmbedding.STATIC, dn.UseEmbedding.NON_STATIC]
kernel_functions = ['lecun_uniform', 'glorot_uniform']
for kernel_function in kernel_functions:
    for use_embedding in use_embeddings:
        exp.pp_data.embedding_type = dn.EmbeddingType.GLOVE_6B
        exp.pp_data.use_embedding = use_embedding
        exp.pp_data.word_embedding_custom_file = ''
        exp.pp_data.load_dataset_type = dn.LoadDataset.TRAIN_DATA_MODEL

        we_file_name = 'ET_' + str(embedding_type.value) + '_UE_' + str(use_embedding.value) +\
                       '_EF_' + word_embedding_custom_file.split('.')[0] + '_glove6B300d_' + kernel_function

        generate_model(exp, 'exp9_' + we_file_name[0:13] + we_file_name[18:30] + '_2640', we_file_name, 
                       kernel_function)

Initializer experiment 2.3 (model SMHD_ml_gl_2640)
Set: kernel_initializer=glorot_uniform=xavier_uniform, dataset=SMHD_2640 multi-label
Loading data... /home/vanessa/PycharmProjects/RecurrentNetworks/tokenizers/anx_dep_multilabel/SMHD_TR_2640_VS_5000_TF_0_DF_0_RSW_F_IT_1_RP_F
Preprocess data... /home/vanessa/PycharmProjects/RecurrentNetworks/dataset/anx_dep_multilabel/SMHD_train_2640.df
Load Pre-train embeddings
Load data - Ini: 2020-03-11 16:37:52	End: 2020-03-11 16:38:16	Total: 0:00:24.774836
Training using single GPU or CPU..
Train on 2200 samples, validate on 2200 samples
Epoch 1/32
2200/2200 [==============================] - 567s 258ms/step - loss: 0.6773 - acc: 0.5908 - val_loss: 0.6719 - val_acc: 0.6000
Epoch 2/32
2200/2200 [==============================] - 536s 244ms/step - loss: 0.6701 - acc: 0.6003 - val_loss: 0.6612 - val_acc: 0.6029
Epoch 3/32
2200/2200 [==============================] - 531s 241ms/step - loss: 0.6590 - acc: 0.6032 - val_loss: 0.6446 - val_acc: 0.6289
Epo

Epoch 17/32
2200/2200 [==============================] - 529s 240ms/step - loss: 0.6272 - acc: 0.6453 - val_loss: 0.6499 - val_acc: 0.6156
Epoch 18/32
2200/2200 [==============================] - 519s 236ms/step - loss: 0.6240 - acc: 0.6456 - val_loss: 0.6518 - val_acc: 0.6191
Epoch 19/32
2200/2200 [==============================] - 525s 239ms/step - loss: 0.6237 - acc: 0.6491 - val_loss: 0.6562 - val_acc: 0.6124
Epoch 20/32
2200/2200 [==============================] - 524s 238ms/step - loss: 0.6262 - acc: 0.6338 - val_loss: 0.6568 - val_acc: 0.6153
Epoch 21/32
2200/2200 [==============================] - 521s 237ms/step - loss: 0.6261 - acc: 0.6432 - val_loss: 0.6504 - val_acc: 0.6233
Epoch 22/32
2200/2200 [==============================] - 521s 237ms/step - loss: 0.6234 - acc: 0.6514 - val_loss: 0.6456 - val_acc: 0.6232
Epoch 23/32
2200/2200 [==============================] - 518s 235ms/step - loss: 0.6297 - acc: 0.6444 - val_loss: 0.6461 - val_acc: 0.6208
Epoch 24/32
2200/2200 [====

2200/2200 [==============================] - 497s 226ms/step - loss: 0.6631 - acc: 0.6027 - val_loss: 0.6685 - val_acc: 0.6008
Epoch 24/32
2200/2200 [==============================] - 498s 226ms/step - loss: 0.6636 - acc: 0.6011 - val_loss: 0.6691 - val_acc: 0.6033
Epoch 25/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6619 - acc: 0.6073 - val_loss: 0.6574 - val_acc: 0.6111
Epoch 26/32
2200/2200 [==============================] - 496s 225ms/step - loss: 0.6565 - acc: 0.6083 - val_loss: 0.6422 - val_acc: 0.6289
Epoch 27/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6575 - acc: 0.6068 - val_loss: 0.6380 - val_acc: 0.6344
Epoch 28/32
2200/2200 [==============================] - 497s 226ms/step - loss: 0.6539 - acc: 0.6114 - val_loss: 0.6533 - val_acc: 0.6155
Epoch 29/32
2200/2200 [==============================] - 498s 226ms/step - loss: 0.6541 - acc: 0.6092 - val_loss: 0.6624 - val_acc: 0.6098
Epoch 30/32
2200/2200 [================